# Setup

In [2]:
%reload_ext autoreload

In [3]:
# Silence WARNING:root:The use of `check_types` is deprecated and does not have any effect.
# https://github.com/tensorflow/probability/issues/1523
import logging

logger = logging.getLogger()


class CheckTypesFilter(logging.Filter):
    def filter(self, record):
        return "check_types" not in record.getMessage()


logger.addFilter(CheckTypesFilter())

In [1]:

import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=3)
import scipy.stats
import einops
from functools import partial

from functools import partial
from collections import namedtuple
import itertools
from itertools import repeat
from time import time
import chex


import jax
import jax.random as jr
import jax.numpy as jnp
from jax import vmap, grad, jit, lax
from jax import numpy as jnp
import jax.scipy as jsp

from flax.core import freeze, unfreeze
from flax import linen as nn
import flax

import jaxopt
import optax
import tensorflow as tf
import tensorflow_datasets as tfds

import torch
from torch.utils.data import TensorDataset, DataLoader


import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset
from PIL import Image
import torchvision.transforms as T

#jax.config.update("jax_enable_x64", False)



2022-11-09 04:21:30.355836: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-11-09 04:21:30.392375: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-09 04:21:31.196091: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-11-09 04:21:31.196229: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib


In [2]:
import os 
cpu_count = os.cpu_count()
print(cpu_count)

# Run jax on multiple CPU cores
# https://github.com/google/jax/issues/5506
# https://stackoverflow.com/questions/72328521/jax-pmap-with-multi-core-cpu
import os 
#os.environ["XLA_FLAGS"] = '--xla_force_host_platform_device_count=90'

import jax
print(jax.devices())

96
[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1), TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1), TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1), TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0), TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]


# Library code

In [6]:
%pwd

'/home/kpmurphy/github/label-shift/tta'

In [7]:
import tta
from tta.utils import *
print(Dataset)

from tta.datasets import *
print(MultipleDomainDataset)

from tta.datasets.chexpert import *
print(MultipleDomainCheXpert)


<class 'tta.utils.Dataset'>
<class 'tta.datasets.MultipleDomainDataset'>
<class 'tta.datasets.chexpert.MultipleDomainCheXpert'>


# Load the labels

In [100]:
%pwd

'/home/kpmurphy/github/label-shift/tta'

In [69]:
from pathlib import Path
root = '/home/kpmurphy/data/CheXpert'
root = Path(root)
labels = pd.read_csv(root / "labels.csv", index_col="image_id")




In [24]:

labels.head()

,Unnamed: 0,NO_FINDING,ENLARGED_CARDIOMEDIASTINUM,CARDIOMEGALY,AIRSPACE_OPACITY,LUNG_LESION,PULMONARY_EDEMA,CONSOLIDATION,PNEUMONIA,ATELECTASIS,...,EFFUSION,PLEURAL_OTHER,FRACTURE,SUPPORT_DEVICES,patient_id,split,GENDER,AGE_AT_CXR,PRIMARY_RACE,ETHNICITY
image_id,,,,,,,,,,,,,,,,,,,,,
CheXpert-v1.0/train/patient42720/study2/view1_frontal.jpg,0,3,1,3,3,3,1,3,3,3,...,1,3,3,1,patient42720,train,Male,58,White,Non-Hispanic/Non-Latino
CheXpert-v1.0/train/patient42720/study7/view1_frontal.jpg,1,3,3,0,1,3,3,3,3,3,...,1,3,3,1,patient42720,train,Male,58,White,Non-Hispanic/Non-Latino
CheXpert-v1.0/train/patient42720/study8/view1_frontal.jpg,2,3,3,0,1,3,3,3,3,3,...,1,3,3,1,patient42720,train,Male,58,White,Non-Hispanic/Non-Latino
CheXpert-v1.0/train/patient42720/study6/view1_frontal.jpg,3,3,3,3,3,3,1,3,3,3,...,1,3,3,1,patient42720,train,Male,58,White,Non-Hispanic/Non-Latino
CheXpert-v1.0/train/patient42720/study1/view1_frontal.jpg,4,3,3,3,3,3,1,3,3,3,...,1,3,3,1,patient42720,train,Male,58,White,Non-Hispanic/Non-Latino


In [27]:
len(labels)

190499

# Joint the embeddings with the labels

In [104]:
def extract_labels(root, max_rows=0):
    labels = pd.read_csv(root / "labels.csv", index_col="image_id")

    # Extract subset of rows for which all labels are available
    labels = labels.loc[labels["PNEUMONIA"].isin({1, 3})]
    labels = labels.loc[labels["EFFUSION"].isin({1, 3})]
    labels = labels.loc[labels["GENDER"] != "Unknown"]

    if max_rows == 0:
        max_rows = len(labels)

    columns = ["PNEUMONIA", "EFFUSION", "GENDER"]
    for t in columns:
        code, uniques = pd.factorize(labels[t], sort=True)
        print(t, code, uniques) #1->0, 3->1, female->0, male->1
        labels[t] = code
    
    m = np.median(labels["AGE_AT_CXR"])
    print('median age ', m)
    labels["AGE_QUANTIZED"] = (labels["AGE_AT_CXR"] > m)
    columns.append("AGE_QUANTIZED")

    YZ = labels[columns].to_numpy()
    YZ = YZ[:max_rows]
    return YZ, labels, columns

In [106]:
YZ, labels, columns = extract_labels(root, max_rows=0)
print(YZ.shape)

PNEUMONIA [1 1 1 ... 0 0 0] Int64Index([1, 3], dtype='int64')
EFFUSION [0 0 0 ... 0 0 0] Int64Index([1, 3], dtype='int64')
GENDER [1 1 1 ... 1 0 0] Index(['Female', 'Male'], dtype='object')
median age  62.0
(139907, 4)


In [119]:
print(type(YZ))

<class 'numpy.ndarray'>


In [105]:
def extract_features(root, labels, max_rows=20):
    datastore = np.load(root / "embeddings.npz")
    if max_rows == 0:
        max_rows = len(labels)
    ndims = 1376
    X = np.zeros((max_rows, ndims))
    i = 0
    for fname in labels.index:
        x = datastore[fname]
        i += 1
        if i >= max_rows: break
        X[i,:] = x
    return X

In [126]:
%%time
X = extract_features(root, labels, max_rows=0)
print(X.shape)



(139907, 1376)
CPU times: user 15min 47s, sys: 4.41 s, total: 15min 51s
Wall time: 15min 49s


In [123]:
print(columns)

['PNEUMONIA', 'EFFUSION', 'GENDER', 'AGE_QUANTIZED']


In [127]:
np.savez(root / 'data_matrix.npz', X=X, YZ=YZ, columns=columns)


# Load pre-computed data matrix

In [2]:

from pathlib import Path
root = '/home/kpmurphy/data/CheXpert'
root = Path(root)

data = np.load(root / 'data_matrix.npz', allow_pickle=True)
print(data.files)
print(data['X'].shape)
print(data['YZ'].shape)
print(data['columns'])

['X', 'YZ', 'columns']
(139907, 1376)
(139907, 4)
['PNEUMONIA' 'EFFUSION' 'GENDER' 'AGE_QUANTIZED']


# Fit a logistic regression model with sklean

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline

In [28]:
X = data['X']
ndx = np.where(data['columns'] == 'EFFUSION')[0][0]
Y = np.array(data['YZ'][:,ndx], dtype=int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


N_train  = X_train.shape[0]
N_test  = X_test.shape[0]
print([N_train, N_test])

classifier = Pipeline([
        ('standardscaler', StandardScaler()),
        #('poly', PolynomialFeatures(degree=2)), 
        ('logreg', LogisticRegression(random_state=0, max_iter=500, C=10, solver='sag', multi_class='multinomial'))
])

classifier = LogisticRegression(random_state=0, max_iter=500, C=10, solver='sag', multi_class='multinomial')


[111925, 27982]


In [29]:


#N = 100
N  = N_train
XX = X_train[:N]
YY = Y_train[:N]




In [30]:
%%time
classifier.fit(XX, YY)

In [27]:

probs = classifier.predict_proba(X_test)

y_pred = jnp.argmax(probs, axis=1)
y_pred2 = classifier.predict(X_test)
assert np.allclose(y_pred, y_pred2)

y_true = Y_test
acc = sklearn.metrics.accuracy_score(y_true, y_pred)
print(acc)



0.6214352083482239


# SKAX

In [1]:

import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
import einops

from functools import partial
from collections import namedtuple
import jax
import jax.random as jr
import jax.numpy as jnp
from jax import vmap, grad, jit
#import jax.debug
import itertools
from itertools import repeat
from time import time
import chex
import typing

import jax
from typing import Any, Callable, Sequence
from jax import lax, random, numpy as jnp
from flax.core import freeze, unfreeze
from flax import linen as nn
import flax
from flax.training import train_state

import optax
import distrax

        

2022-11-09 05:07:15.415404: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-11-09 05:07:15.449361: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-09 05:07:16.178259: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-11-09 05:07:16.178429: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib


In [ ]:

def logprior_fn(params, sigma):
    # log p(params)
    leaves, _ = jax.tree_util.tree_flatten(params)
    flat_params = jnp.concatenate([jnp.ravel(a) for a in leaves])
    return jnp.sum(distrax.Normal(0, sigma).log_prob(flat_params))

@partial(jax.jit, static_argnums=(1,2))
def get_batch_train_ixs(key, num_train, batch_size):
    # return indices of training set in a random order
    steps_per_epoch = num_train // batch_size
    batch_ixs = jax.random.permutation(key, num_train)
    batch_ixs = batch_ixs[:steps_per_epoch * batch_size]
    batch_ixs = batch_ixs.reshape(steps_per_epoch, batch_size)
    return batch_ixs


class LogRegNetwork(nn.Module):
    nclasses: int

    @nn.compact
    def __call__(self, x):
        logits = nn.Dense(self.nclasses)(x)
        return logits

class MLPNetwork(nn.Module):
  nfeatures_per_layer: Sequence[int]

  @nn.compact
  def __call__(self, inputs):
    x = inputs
    nlayers = len(self.nfeatures_per_layer)
    for i, feat in enumerate(self.nfeatures_per_layer):
      x = nn.Dense(feat, name=f'layers_{i}')(x)
      if i != (nlayers - 1):
        #x = nn.relu(x)
        x = nn.gelu(x)
    return x

In [ ]:


class NeuralNetClassifier:
    def __init__(self, network, key, nclasses, *,  l2reg=1e-5, standardize = True,
                optimizer = 'adam+warmup', batch_size=128, max_iter=100, num_epochs=10, print_every=0):
        # optimizer is one of {'adam+warmup'} or an optax object
        self.nclasses = nclasses
        self.network = network
        self.standardize = standardize
        self.max_iter = max_iter
        self.num_epochs = num_epochs
        self.optimizer = optimizer
        self.batch_size = batch_size
        self.l2reg = l2reg
        self.print_every = print_every
        self.params = None # must first call fit
        self.key = key

    def predict(self, X):
        if self.params is None:
            raise ValueError('need to call fit before predict')
        if self.standardize:
            X = X - self.mean
            X = X / self.std
        return jax.nn.softmax(self.network.apply(self.params, X))

    def fit(self, X, y):
        if self.standardize:
            self.mean = jnp.mean(X, axis=0)
            self.std = jnp.std(X, axis=0) + 1e-5 
            X = X - self.mean
            X = X / self.std
        if self.params is None:
            nfeatures = X.shape[1]
            x = jr.normal(self.key, (nfeatures,)) # single random input 
            self.params = self.network.init(self.key, x) 
        ntrain = X.shape[0]
        if isinstance(self.optimizer, str) and (self.optimizer.lower() == "adam+warmup"):
            total_steps = self.num_epochs*(ntrain//self.batch_size)  
            warmup_cosine_decay_scheduler = optax.warmup_cosine_decay_schedule(
                init_value=1e-3, peak_value=1e-1, warmup_steps=int(total_steps*0.1),
                decay_steps=total_steps, end_value=1e-3)
            self.optimizer = optax.adam(learning_rate=warmup_cosine_decay_scheduler)
            return self.fit_optax(self.key, X, y)
        else:
            #return self.fit_jaxopt(self.key, X, y)
            return self.fit_optax(self.key, X, y)

    def fit_optax(self, key, X, y):
        # Based on https://github.com/google/flax/blob/main/examples/mnist/train.py
        # https://github.com/google/flax/blob/main/examples/vae/train.py
        sigma = np.sqrt(1/self.l2reg)
        ntrain = X.shape[0]
 
        @jax.jit
        def train_step(key, state, Xb, yb):
            # Computes gradients, loss and accuracy for a single batch.
            # loss = -1/N [ (sum_n log p(yn|xn, theta)) + log p(theta) ]
            def loss_fn(params):
                logits = state.apply_fn({'params': params}, Xb)
                loglik = jnp.mean(distrax.Categorical(logits).log_prob(yb))
                logjoint = loglik + (1/ntrain)*logprior_fn(params, sigma)
                return -logjoint
            loss, grads = jax.value_and_grad(loss_fn)(state.params)
            return loss, state.apply_gradients(grads=grads)

        def train_epoch(key, state):
            key, sub_key = jr.split(key)
            batch_ixs = get_batch_train_ixs(sub_key, ntrain, self.batch_size)
            num_batches = len(batch_ixs)
            key, sub_key = jr.split(key)
            keys = jax.random.split(sub_key, num_batches)    
            total_loss = 0
            for key, batch_ix in zip(keys, batch_ixs):
                X_batch, y_batch = X[batch_ix], y[batch_ix]
                loss, state = train_step(key, state, X_batch, y_batch)
                total_loss += loss
            return total_loss.item(), state


        # main loop
        state = train_state.TrainState.create(
            apply_fn=self.network.apply, params=self.params['params'], tx=self.optimizer)
        for epoch in range(self.num_epochs):
            key, sub_key = jr.split(key)
            train_loss, state = train_epoch(sub_key, state)
            if (self.print_every > 0) and (epoch % self.print_every == 0):
                print('epoch {:d}, train loss {:0.3f}'.format(epoch, train_loss))

        self.params = {'params': state.params}

        
      

# Make shifted datasets for each domain


In [ ]:
root = Path("/home/kpmurphy/data/CheXpert")
dataset_y_column = "EFFUSION"
dataset_z_column = "GENDER"
dataset_use_embedding = True
train_domains_set = [9]
target_domain_count = 512

import random
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
key = jax.random.PRNGKey(seed)
generator = torch.Generator().manual_seed(seed)


dataset = MultipleDomainCheXpert(root, generator, dataset_y_column, dataset_z_column, dataset_use_embedding, train_domains_set, target_domain_count)